# Model definition
The world frame is defined with $x$ to the right and $y$ up. Rotations are CCW positive.
Consider a 2D birotor with mass $m_{b}$, moment of inertia $I_{b}$ about its center of mass (CoM), which is exactly between the rotors. The distance between the rotors is $L_{b}$. At the center of mass of the base, a revolute joint is attached which actuates a manipulator made up of one member of length $L_{m}$. Its mass $m_{m}$ is concentrated at $\frac{1}{2}L_{m}$ and its moment of inertia is $I_{m}$ about the CoM. 
The pitch of the birotor $\theta_{b}$ is defined 0 in the horizontal orientation, and the nominal configuration of the manipulator is defined to the right (positive $x$), where the revolute joint rotation $q$ is defined 0.

First, we set up the kinematics function of the manipulator.

Then, we set up the equations of motion by using the Euler-Lagrange method.

In [23]:
import sympy as sp
import numpy as np
from sympy import symbols, Function, cos, sin, diff
from sympy.physics.vector import init_vprinting

init_vprinting() # Pretty printing with sympy

# Time variable
t = symbols('t')

# Other variables
mb, mm, Ib, Im, Lb, Lm, g = symbols('m_b m_m I_b I_m L_b L_m g')

# Generalized coordinates as functions of time
xb = Function('x_b')(t)
yb = Function('y_b')(t)
thetab = Function('theta_b')(t)
q = Function('q')(t)

# Derivatives
xb_dot = diff(xb, t)
yb_dot = diff(yb, t)
thetab_dot = diff(thetab, t)
q_dot = diff(q, t)

xb_ddot = diff(xb_dot, t)
yb_ddot = diff(yb_dot, t)
thetab_ddot = diff(thetab_dot, t)
q_ddot = diff(q_dot, t)

# End-effector coordinates
xe = Function('x_e')(t)
ye = Function('y_e')(t)
thetae = Function('theta_e')(t)

xe_dot = diff(xe, t)
ye_dot = diff(ye, t)
thetae_dot = diff(thetae, t)

In [75]:
# Deriving the kinematic equation
xe = xb + Lm*cos(thetab+q)
ye = yb + Lm*sin(thetab+q)
thetae = thetab + q
k = sp.Matrix([xe, ye, thetae]) # kinematic equation
sp.pprint(k)

# Jacobian
J_A = k.jacobian([xb, yb, thetab, q])
print(J_A.shape)

⎡Lₘ⋅cos(q(t) + θ_b(t)) + x_b(t)⎤
⎢                              ⎥
⎢Lₘ⋅sin(q(t) + θ_b(t)) + y_b(t)⎥
⎢                              ⎥
⎣        q(t) + θ_b(t)         ⎦
(3, 4)


### Controller
Now that we have the differential kinematics, we can employ a second-order inverse kinematics algorithm to get references for the controllable states. We follow the formulation of Arleo (2013).

In [81]:
J_A_controlled = J_A[:,0:2].row_join(J_A[:,3])
J_A_uncontrolled = J_A[:,2]

# Set the parameters
J_A_controlled = J_A_controlled.subs({Lm:1})
J_A_uncontrolled = J_A_uncontrolled.subs({Lm:1})

# CLIK inputs
K = sp.eye(3) # gain matrix

# Forward kinematics function
def forward_kinematics(xi):
    return k.subs({xb:xi[0], yb:xi[1], thetab:xi[2], q:xi[3], Lm:1})

# CLIK law
def CLIK(xe_current, xe_ref, xe_dot_ref, xi_measured, J_A_controlled, J_A_uncontrolled, K):
    J_A_controlled_sub = J_A_controlled.subs({xb:xi_measured[0], yb:xi_measured[1], thetab:xi_measured[2], q:xi_measured[3]})
    J_A_uncontrolled_sub = J_A_uncontrolled.subs({xb:xi_measured[0], yb:xi_measured[1], thetab:xi_measured[2], q:xi_measured[3]})
    return J_A_controlled_sub.inv()*(xe_dot_ref - K*(xe_ref - xe_current)) - J_A_controlled_sub.inv()*J_A_uncontrolled_sub*thetab_dot

# Set some initial conditions
xi_current = sp.Matrix([0, 0, 0, 0])
timestep = 0.01

# References
xe_des = np.array([[1.0], [0.5], [0.0]])  # Desired task-space pose
xe_goal_vel = np.array([[0.0], [0.0], [0.0]])  # Goal velocity

for _ in range(10000):
    xe_current = forward_kinematics(xi_current)
    J_A_controlled_sub = J_A_controlled.subs({xb:xi_current[0], yb:xi_current[1], thetab:xi_current[2], q:xi_current[3]})
    J_A_uncontrolled_sub = J_A_uncontrolled.subs({xb:xi_current[0], yb:xi_current[1], thetab:xi_current[2], q:xi_current[3]})
    e = xe_current - xe_des
    e_dot = xe_goal_vel - K*e
    CLIK_output = J_A_controlled_sub.inv()*e_dot - J_A_controlled_sub.inv()*J_A_uncontrolled_sub*0.0

    # Euler integration
    xi_current = xi_current + timestep*sp.Matrix([CLIK_output[0], CLIK_output[1], 0.0, CLIK_output[2]])
    error_norm = (e.T.dot(e).evalf())
    if error_norm.evalf() < 0.001:
        print("Iterations: ", _)
        print("State")
        print(xi_current)
        print("Controlled state velocity references")
        print(CLIK_output)
        print("Task-space error")
        print(e)
        print("Task-space pose")
        print(forward_kinematics(xi_current))
        break

Iterations:  275
State
Matrix([[0], [0.468790722610088], [0], [0]])
Controlled state velocity references
Matrix([[0], [0.0315245226160727], [0]])
Task-space error
Matrix([[0], [-0.0315245226160727], [0]])
Task-space pose
Matrix([[1], [0.468790722610088], [0]])


In [ ]:
# Dynamics
# Deriving the kinetic energy
T_b = mb*(xb_dot**2 + yb_dot**2)/2 + Ib*thetab_dot**2/2
T_m = mm*(xb_dot**2 + yb_dot**2 + (Lm*q_dot**2)/4 + (Lm*q_dot*thetab_dot)/2 + (Lm*thetab_dot**2)/4.)/2 + Im*(q_dot+thetab_dot)**2/2

# Deriving the potential energy
U_b = mb*g*yb
U_m = mm*g*yb + mm*g*Lm*sin(q+thetab)/2

# Deriving the Lagrangian
L = T_b + T_m - U_b - U_m

# Deriving the equations of motion
eq1 = diff(diff(L, xb_dot), t) - diff(L, xb)
eq2 = diff(diff(L, yb_dot), t) - diff(L, yb)
eq3 = diff(diff(L, thetab_dot), t) - diff(L, thetab)
eq4 = diff(diff(L, q_dot), t) - diff(L, q)

sp.pprint("Equations of motion")
sp.pprint(eq1)
sp.pprint(eq2)
sp.pprint(eq3)
sp.pprint(eq4)

Equations of motion
     2              2       
    d              d        
m_b⋅───(x(t)) + mₘ⋅───(x(t))
      2              2      
    dt             dt       
                    2              2       
                   d              d        
g⋅m_b + g⋅mₘ + m_b⋅───(y(t)) + mₘ⋅───(y(t))
                     2              2      
                   dt             dt       
                                                                               ↪
                                                                               ↪
                                                                               ↪
                   ⎛   2             2       ⎞                                 ↪
                   ⎜  d             d        ⎟                                 ↪
                Iₘ⋅⎜2⋅───(q(t)) + 2⋅───(θ(t))⎟                              mₘ ↪
     2             ⎜    2             2      ⎟                                 ↪
    d              ⎝  dt            dt       ⎠  

# Controller
Now that we have the EoMs of the aerial manipulator, we can set up the 